In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.python.keras.models import Sequential


from tensorflow.keras.datasets import cifar10
import numpy as np

In [3]:
def create_denoising_autoencoder():
    input_img = Input(shape=(32, 32, 3))

    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    return autoencoder



In [4]:
# Cargar el dataset CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalizar los datos
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [5]:
autoencoder = create_denoising_autoencoder()

# Generar datos ruidosos
noise_factor = 0.5
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_train_noisy = np.clip(X_train_noisy, 0., 1.)

X_test_noisy = X_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)

# Entrenar el autoencoder
autoencoder.fit(X_train_noisy, X_train, epochs=30, batch_size=256, shuffle=True, validation_data=(X_test_noisy, X_test))

Epoch 1/30
196/196 [==============================] - 57s 281ms/step - loss: 0.6031 - val_loss: 0.5860
Epoch 2/30
196/196 [==============================] - 56s 286ms/step - loss: 0.5832 - val_loss: 0.5824
Epoch 3/30
196/196 [==============================] - 57s 292ms/step - loss: 0.5810 - val_loss: 0.5810
Epoch 4/30
196/196 [==============================] - 52s 266ms/step - loss: 0.5798 - val_loss: 0.5802
Epoch 5/30
196/196 [==============================] - 51s 259ms/step - loss: 0.5788 - val_loss: 0.5793
Epoch 6/30
196/196 [==============================] - 59s 300ms/step - loss: 0.5782 - val_loss: 0.5786
Epoch 7/30
196/196 [==============================] - 51s 258ms/step - loss: 0.5778 - val_loss: 0.5783
Epoch 8/30
196/196 [==============================] - 51s 261ms/step - loss: 0.5774 - val_loss: 0.5779
Epoch 9/30
196/196 [==============================] - 51s 260ms/step - loss: 0.5770 - val_loss: 0.5777
Epoch 10/30
196/196 [==============================] - 51s 258ms/step - l

In [6]:
autoencoder.save('autoencoder.h5')